# Claening the data for ETF sector 

# Read Data

In [5]:
import pandas as pd
options_data = pd.read_csv('../Data/Options_data_v2.csv')
#get Call data
index_names = options_data[options_data['OptionType'] == 'put'].index
options_data.drop(index_names, inplace=True)
#filter sector
other_sector = options_data[options_data['Sector'] != 'ETF'].index
options_data.drop(other_sector, inplace=True)
#remove NA
options_data.dropna(axis=0, inplace=True)
#reset index
options_data.reset_index(inplace=True)

In [6]:
options_data.head(3)                                                                                                              

,index,Unnamed: 0,OptionType,Time,Sym,Expiration,Strike,StockPrice,Date,Spent,Sector,ExpirationDate,ChainLocation,Spent/contract,Spent/share,break_even,Contract period in hrs,Contract period in days
0,3,3,call,09:43:00.0000000,GLD,08-19 M,146.0,141.01,2019-08-09,23000,ETF,2019-08-16,OTM,230.0,2.3,148.3,174.283056,7
1,11,11,call,10:32:00.0000000,SLV,12-19 Q,17.5,15.93,2019-08-09,250000,ETF,2019-12-20,OTM,2500.0,25.0,42.5,3197.466389,133
2,14,14,call,10:37:00.0000000,USO,09/27-19 W,12.5,11.22,2019-08-09,28000,ETF,2019-09-27,OTM,280.0,2.8,15.3,1181.383056,49


In [8]:
top10 = options_data.groupby('Sym').count().sort_values('OptionType', ascending=False).reset_index()
top10[['Sym','OptionType']].head()

,Sym,OptionType
0,SPY,2838
1,QQQ,666
2,IWM,585
3,EEM,326
4,GLD,281


In [9]:
sym = top10[['Sym']].iloc[:10, 0].tolist()
#filter by Sector
other_symbols = options_data[~options_data['Sym'].isin(sym)].index
options_data.drop(other_symbols, inplace=True)
sym

['SPY', 'QQQ', 'IWM', 'EEM', 'GLD', 'XLF', 'FXI', 'XOP', 'GDX', 'USO']

In [13]:
def get_stock_data(sym):
  sector_data = pd.DataFrame()
  for i in sym:
    sym_data = pd.read_csv('../Data/Stock/'+ i +'.csv')
    sym_data['Sym'] = i
    sector_data = pd.concat([sector_data, sym_data])
  return sector_data

In [14]:
def get_attention_trend_data(sym):
  attention_data = pd.DataFrame()
  for i in sym:
    sym_data = pd.read_csv('../Data/Attention/'+ i +'_attention.csv')
    sym_data['Sym'] = i
    attention_data = pd.concat([attention_data, sym_data])
  return attention_data

In [17]:
stock_data = get_stock_data(sym)
attention_data =get_attention_trend_data(sym)

In [18]:
attention_data.head()

,date,attention,Sym
0,2017-01-01,63.0,SPY
1,2017-01-02,60.0,SPY
2,2017-01-03,58.0,SPY
3,2017-01-04,60.0,SPY
4,2017-01-05,56.0,SPY


In [19]:
stock_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Sym
0,2017-11-01,258.040009,258.429993,257.070007,257.489990,241.419128,54202700,SPY
1,2017-11-02,257.410004,257.750000,256.190002,257.589996,241.512909,56449500,SPY
2,2017-11-03,257.769989,258.500000,257.299988,258.450012,242.319229,59589700,SPY
3,2017-11-06,258.299988,259.000000,258.220001,258.850006,242.694260,49652600,SPY
4,2017-11-07,258.970001,259.350006,258.089996,258.670013,242.525497,57502200,SPY


In [20]:
stock_data['ExpirationDate'] = stock_data['Date'].astype('datetime64[ns]')
attention_data['ExpirationDate'] = attention_data['date'].astype('datetime64[ns]')
stock_data = stock_data.drop('Date', axis=1)
attention_data = attention_data.drop('date', axis=1)
options_data['ExpirationDate'] =options_data['ExpirationDate'].astype('datetime64[ns]')

In [21]:
ETF_data = stock_data.merge(options_data,on=['ExpirationDate', 'Sym'],how='inner')

In [22]:
ETF_data = ETF_data.merge(attention_data,on=['ExpirationDate', 'Sym'],how='inner')

In [23]:
ETF_data.head(-5)

,Open,High,Low,Close,Adj Close,Volume,Sym,ExpirationDate,index,Unnamed: 0,...,Date,Spent,Sector,ChainLocation,Spent/contract,Spent/share,break_even,Contract period in hrs,Contract period in days,attention
0,256.619995,257.220001,255.630005,256.440002,240.434677,80811500,SPY,2017-11-15,42821,42821,...,2017-11-15,16000,ETF,OTM,160.0,1.6,258.6,6.674722,0,43.0
1,256.619995,257.220001,255.630005,256.440002,240.434677,80811500,SPY,2017-11-15,42830,42830,...,2017-11-15,36000,ETF,OTM,360.0,3.6,260.6,5.980000,0,43.0
2,256.619995,257.220001,255.630005,256.440002,240.434677,80811500,SPY,2017-11-15,42909,42909,...,2017-11-14,65000,ETF,ATM,650.0,6.5,264.0,31.097778,1,43.0
3,256.619995,257.220001,255.630005,256.440002,240.434677,80811500,SPY,2017-11-15,42920,42920,...,2017-11-14,65000,ETF,ITM,650.0,6.5,264.0,30.214722,1,43.0
4,256.619995,257.220001,255.630005,256.440002,240.434677,80811500,SPY,2017-11-15,42921,42921,...,2017-11-14,90000,ETF,ITM,900.0,9.0,266.5,30.202778,1,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5666,97.839996,98.559998,96.639999,97.599998,97.599998,3009613,USO,2019-09-20,1704,1704,...,2019-07-18,68000,ETF,ITM,680.0,6.8,18.3,1541.433056,64,42.0
5667,97.839996,98.559998,96.639999,97.599998,97.599998,3009613,USO,2019-09-20,2172,2172,...,2019-07-12,30000,ETF,OTM,300.0,3.0,16.5,1683.883056,70,42.0
5668,92.559998,94.720001,92.400002,93.279999,93.279999,3215050,USO,2019-09-27,14,14,...,2019-08-09,28000,ETF,OTM,280.0,2.8,15.3,1181.383056,49,41.0
5669,92.559998,94.720001,92.400002,93.279999,93.279999,3215050,USO,2019-09-27,17,17,...,2019-08-09,14000,ETF,OTM,140.0,1.4,13.9,1181.249722,49,41.0


In [24]:
ETF_data['Profit'] = (ETF_data['break_even'] < ETF_data['Close']) 
S = ETF_data[ETF_data['Profit'] == True]

In [25]:
print("Total number of profitable options: ",S.shape[0])
print("Total number of data entries: ",ETF_data.shape[0])

Total number of profitable options:  397
Total number of data entries:  5676


In [26]:
ETF_data.to_csv('ETF_cleaned.csv')